# Convert quantum results to GPT3 strings

- Count total shots
- Filter errors (eg: '00110', '00000')
- Convert qbit binary string back to index
- Order results by original places index
- Map results to places and convert to probabilities

In [114]:
# JSON data
json_data = {
    "computer": "ionq",  # quantum computer to use
    "qbit_count": 5,     # number of qubits to use
    "places": [          # list of places to visit and their probabilities
        ["Gym", 0.2],
        ["Opera", 0.5],
        ["Rooftop bar", 0.9],
        ["Street fair", 0.0],
        ["Pool", 0.1],
    ]
}
places = json_data['places']

# Quantum computer results
result = {
  "10000": 85,  # 5
  "00001": 832, # 1
  "00100": 4,   # 3
  "00010": 99,  # 2
  "01000": 1,   # 4
  "01100": 2,   # Error
  "00000": 1,   # Error
}

In [115]:
# Total shots
total_shots = sum(result.values())
print(total_shots)

1024


In [116]:
# Detect errors and remove it from the results
def filter_errors(result):
    return {key: value for key, value in result.items() if key.count('1') == 1}

# Clean results
filtered_result = filter_errors(result)
print(filtered_result)

{'10000': 85, '00001': 832, '00100': 4, '00010': 99, '01000': 1}


In [117]:
# Convert binary strings to places (e.g. '00010' -> 1)
def convert_binary_to_index(binary_string):
    if '1' in binary_string:
        return len(binary_string) - binary_string.index('1') - 1
    else:
        return -1

index2 = convert_binary_to_index('00100')
print(index2)

2


In [118]:
# Order results using the original index
def order_results(result):
    ordered_result = []
    for key, value in result.items():
        index = convert_binary_to_index(key)
        ordered_result.append([index, key, value])
    ordered_result.sort()
    return ordered_result

ordered_result = order_results(filtered_result)
print(ordered_result)

[[0, '00001', 832], [1, '00010', 99], [2, '00100', 4], [3, '01000', 1], [4, '10000', 85]]


In [120]:
# Convert qbit keys and shots to places and probabilities (e.g. [1, '00010', 85] -> ['Opera', 0.085])
def convert_to_places(result, places, total_shots):
    readable_result = []
    for line in result:
        place = places[line[0]][0]
        probability = line[2] / total_shots
        readable_result.append([place, probability])
    return readable_result

processed_results = convert_to_places(ordered_result, places, total_shots)
print(processed_results)

[['Gym', 0.8125], ['Opera', 0.0966796875], ['Rooftop bar', 0.00390625], ['Street fair', 0.0009765625], ['Pool', 0.0830078125]]


In [126]:
def convert_probability_to_likelyhood(probability):
    ''' Method to convert probabilities to likelyhoods strings.'''
    if 0 <= probability < 0.25:
        return "wasn't at the "
    elif 0.25 <= probability < 0.75:
        return "may have gone to the "
    elif 0.75 <= probability <= 1.0:
        return "likely went to the "
    else:
        raise ValueError('Probability must be between 0 and 1.')

# Convert probabilities to words
readable_result = []
for line in processed_results:
    likelyhood = convert_probability_to_likelyhood(line[1])
    place = line[0]
    readable_result.append(likelyhood + place)
print(readable_result)

['likely went  the Gym', "wasn't at  the Opera", "wasn't at  the Rooftop bar", "wasn't at  the Street fair", "wasn't at  the Pool"]
